In [1]:
# Momentum Balance Diagnostics
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
sd = 9 # Sill depth
Diag = xr.open_dataset('/Volumes/Extreme SSD/MITgcm outputs/qf700_drag000/UmomDiag_' + str(format(sd,'03d')) + '.nc')
SrfDiag = xr.open_dataset('/Volumes/Extreme SSD/MITgcm outputs/qf700_drag000/SrfDiag_' + str(format(sd,'03d')) + '.nc')
Grid = xr.open_dataset('/Volumes/Extreme SSD/MITgcm outputs/qf700_drag000/grid_' + str(format(sd,'03d')) + '.nc')
State = xr.open_dataset('/Volumes/Extreme SSD/MITgcm outputs/qf700_drag000/state_' + str(format(sd,'03d')) + '.nc')

Ut = Diag.TOTUTEND.data #Tendency of Zonal Component of Velocity (m/s/day) T*Zmd*Y*Xp1
Udis = Diag.Um_Diss.data # U momentum tendency from Dissipation (m/s^2) T*Zmd*Y*Xp1
Uadv = Diag.Um_Advec.data # U momentum tendency from Advection terms (m/s^2) T*Zmd*Y*Xp1
Uext = Diag.Um_Ext.data # U momentum from External Forcing (m/s^2) T*Zmd*Y*Xp1
Uph = Diag.Um_dPHdx.data # U momentum from Hydrostatic Pressure grad (m/s^2) T*Zmd*Y*Xp1
Uab = Diag.AB_gU.data # U momentum from Adam-Bashforth (m/s^2) T*Zmd*Y*Xp1
Ucori = Diag.Um_Cori.data # U momentum tendency from Coriolis term (m/s^2) T*Zmd*Y*Xp1
UvisrI = Diag.VISrI_Um.data # Vertical Viscous Flux of U momentum (Implicit part) (m^4/s^2)  T*Zld*Y*Xp1
Uimp = Diag.Um_ImplD.data # U momentum tendency from Dissipation (Implicit part) (m/s^2) T*Zmd*Y*Xp1
SidDrag = Diag.USidDrag.data # U momentum tendency from Bottom Drag (m/s^2) T*Zmd*Y*Xp1
BotDrag = Diag.UBotDrag.data # U momentum tendency from Side Drag (m/s^2) T*Zmd*Y*Xp1

Etan = SrfDiag.ETAN.data # Surface Height Anomaly (m) T*1*Y*X


rAw = Grid.rAw.data # r-face area at U point (Y*Xp1)
drF = Grid.drF.data # r cell face separation (Z)
HFacW = Grid.HFacW.data # vertical fraction of open cell at West face (Z*Y*Xp1)
dxC = Grid.dxC.data # x cell center separation (Y*Xp1)


Eta = State.Eta.data # free-surface_r-anomaly (m) (T-1)*Y*X


In [6]:
# Set up Gravity, Time and locations
g, t, z, y, x = 9.81, 150, -1, 34, 30

dudt = Ut[t, z, y, x] / 86400
#pg_barotropic = g * (Eta[t+1, y, x] - Eta[t+1, y, x-1]) / dxC[y, x]
pg_barotropic = g * (Etan[t, 0, y, x] - Etan[t, 0, y, x-1]) / dxC[y, x]
pg_baroclinic = Uph[t, z, y, x]
adv = Uadv[t, z, y, x]
dis = Udis[t, z, y, x]
ext = Uext[t, z, y, x]
ab = Uab[t, z, y, x]
visr = (UvisrI[t, z+1, y, x] - UvisrI[t, z, y, x]) / (rAw[y, x] * drF[z] * HFacW[z, y, x])
vimp = Uimp[t, z, y, x]
botdrag = BotDrag[t, z, y, x]
dudt, pg_barotropic, pg_baroclinic, adv, dis, ext, ab, vimp


(3.341750750594117e-07,
 -4.7064608088301515e-06,
 -4.222444e-06,
 -1.5049676e-07,
 1.4322542e-09,
 0.0,
 -3.0941676e-11,
 3.515739e-10)

In [20]:
dudt - pg_baroclinic - adv - dis - ext - ab - vimp + pg_barotropic

-1.098049041943265e-09

In [ ]:
Grid.dxF.data[34,:]

In [ ]:
State.Xp1.data, dxC[y,:]

In [ ]:
Z = State.Zl.data

#np.diff(State.Zl.data)
np.diff(State.Zl.data)

In [ ]:
Diag